In [1]:
import azureml.core
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()

In [3]:
datastore = Datastore.get(ws,'mlteststore')

In [5]:
from azureml.core.compute import ComputeTarget , AmlCompute
compute_name = 'nallaballeramana1'
vm_size = 'Standard_E4as_v4'
compute_target = ws.compute_targets[compute_name]

In [7]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment

aml_run_config = RunConfiguration()
#compute target as defined in Azure machine learning compute section above
aml_run_config.target = compute_target

USE_CURATED_ENV = True

if USE_CURATED_ENV:
    curated_environment = Environment.get(workspace = ws, name = 'AzureML-sklearn-0.24-ubuntu18.04-py37-cpu')
    aml_run_config.environment = curated_environment
else:
    aml_run_config.environment.python.user_managed_dependencies =False

    #add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies =CondaDependencies.create(
        conda_packages = ['Pandas','scikit-learn'],
        pip_packages = ['azureml-sdk', 'azureml-dataset-runtime[fuse,pandas]'],
        pin_sdk_version = False
    )


In [6]:
from azureml.pipeline.steps import PythonScriptStep
#dataprep_source_dir = "./dataprep_src"

entry_point = 'Python_Script.ipynb'

In [10]:
py_script_run_step = PythonScriptStep(
    script_name = entry_point,
    compute_target = compute_target,
    runconfig = aml_run_config,
    allow_reuse = False
)

In [11]:
pipeline_steps = [py_script_run_step]

In [13]:
from azureml.pipeline.core import Pipeline
#building the pipelne
pipeline1 = Pipeline(workspace = ws, steps= [pipeline_steps])

In [23]:
from azureml.core import Experiment
#submit the pipeline to be run
pipeline_run1 = Experiment(ws, 'First_trial_of_custom_pipeline').submit(pipeline1)
pipeline_run1.wait_for_completion(show_output = True)

Submitted PipelineRun 91a28d35-1baf-498f-ac53-38443fcddda8
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/91a28d35-1baf-498f-ac53-38443fcddda8?wsid=/subscriptions/3739307c-4f3c-4bc6-b980-08d7dccd9bbd/resourcegroups/testgroup/workspaces/ml_testing_space&tid=45365ad3-eb1f-4a19-9433-e9528c41f196
PipelineRunId: 91a28d35-1baf-498f-ac53-38443fcddda8
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/91a28d35-1baf-498f-ac53-38443fcddda8?wsid=/subscriptions/3739307c-4f3c-4bc6-b980-08d7dccd9bbd/resourcegroups/testgroup/workspaces/ml_testing_space&tid=45365ad3-eb1f-4a19-9433-e9528c41f196
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: e46f89d7-fb8a-4606-b2ca-ea3614f2b51d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/e46f89d7-fb8a-4606-b2ca-ea3614f2b51d?wsid=/subscriptions/3739307c-4f3c-4bc6-b980-08d7dccd9bbd/resourcegroups/testgroup/workspaces/ml_testing_space&tid=45365ad3-eb1f-4a19-9433-e9528c41f196
StepRun( Python_Scrip

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "{\"NonCompliant\":\"Process 'python' exited with code 1 and error message 'Execution failed. Process exited with status code 1. Error: Traceback (most recent call last):\\n  File \\\"Python_Script.ipynb\\\", line 50, in <module>\\n    \\\"source_hidden\\\": false,\\nNameError: name 'false' is not defined\\n\\n'. Please check the log file 'user_logs/std_log.txt' for more details.\"}\n{\n  \"code\": \"ExecutionFailed\",\n  \"target\": \"\",\n  \"category\": \"UserError\",\n  \"error_details\": [\n    {\n      \"key\": \"exit_codes\",\n      \"value\": \"1\"\n    }\n  ]\n}",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"{\\\"